# Chapter 4. word2vec 속도 개선

3장에서 구현한 CBOW 모델은 말뭉치에 포함된 어휘가 많아지면 계산 속도가 너무 오래 걸립니다.  
이번 장에서는 **word2vec의 속도를 개선**해보겠습니다.  

1. `Embedding` 이라는 새로운 계층 도입
2. `네거티브 샘플링`이라는 새로운 손실 함수 도입

## 4.1 word2vec 개선 (1)

<p align="center"><img src="./master/images/fig 4-1.png" width=500 />

#### 3장에서 구현한 `word2vec`

1. `맥락` 단어 2개를 사용해, $W_{in}$ (입력층 가중치)와의 행렬곱으로 은닉층 계산
2. 1의 결과값을 $W_{out}$ (출력층 가중치)와 행렬곱해서 각 단어의 점수 계산
3. 이 점수에 `Softmax`함수를 적용해 각 단어의 출현 확률 계산
4. 이 확률을 정답 레이블과 비교해서(Cross Entropy Error를 적용해서) 손실 계산

위에서 구현한 `word2vec`은 입력값으로 거대한 말뭉치가 들어오면 단점이 존재  
  
- 입력층의 원핫 표현과 $W_{in}$의 곱 계산 $\rightarrow$ `Embedding` 게층 도입으로 해결
- 은닉층과 $W_{out}$의 곱 및 `Softmax` 계층의 계산 $\rightarrow$ `네거티브 샘플링`이라는 손실함수로 해결
  


<p align="center"><img src="./master/images/fig 4-2.png" width=500 />

### 4.1.1 Embedding 계층

**Embedding 계층이란?**  
가중치 매개변수로부터 '단어 ID에 해당하는 행'을 추출하는 계층  
<p align="left"><img src="./master/images/fig 4-3.png" width=500 />  
  
이 그림처럼 행렬곱을 해서 `h`를 구하는게 아니라, 특정행을 추출

### 4.1.2 Embedding 계층 구현

In [15]:
import numpy as np
W = np.arange(21).reshape(7, 3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [16]:
idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [17]:
params = [W]
print(params)

[array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])]


#### Embedding 계층 forward() 구현

In [18]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

#### Embedding 계층 backward() 구현

<p align="center"><img src="./master/images/fig 4-4.png" width=500 />

In [19]:
    def backward(self, dout):
        dW = self.grads
        dW[...] = 0 # dW의 형상을 유지한채 원소를 0으로 만든다. (dW 자체를 0으로 만드는게 아님)
        dW[self.idx] = dout
        return None

하지만 위처럼 `backward()`를 구현하면 `idx`가 중복될 때 문제가 생김

In [20]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
# idx 중복 문제 해결
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        for i, word_id in enumerate(self, idx):
             dW[word_id] += dout[i] # TODO : 할당이 아니라 왜 더해야하는지 생각해보기
             
        return None

## 4.2 word2vec 개선 (2)

`Softmax` 대신 `네거티브 샘플링`을 이용하면 어휘가 아무리 많아져도 계산량을 낮은 수준에서 일정하게 억제할 수 있게 됨.

### 4.2.1 은닉층 이후 계산의 문제점

<p align="center"><img src="./master/images/fig 4-6.png" width=500 />

#### 은닉충 이후에서 계산이 오래 걸리는 곳
- 은닉층의 뉴런과 가중치 행렬($W_{out})의 곱
- `Softmax` 계층의 계산



### 4.2.2 다중 분류에서 이진 분류로

##### 핵심 아이디어 : **이진 분류(Binary Classification)**  
다중 분류(Multi-class classification)를 `이진 분류`로 근사하는 것이 중요 포인트  

### 4.2.3 시그모이드 함수와 교차 에트로피 오차

이진 분류 문제를 신경망으로 해결할 때는   
- `Sigmoid`를 적용해 확률로 변환
- `Cross Entropy Error`를 손실 함수로 사용

<p align="center"><img src="./master/images/fig 4-9.png" width=700 />

<p align="center"><img src="./master/images/fig 4-10.png" width=700 />

### 4.2.4 다중 분류에서 이진 분류로 (구현)

<p align="center"><img src="./master/images/fig 4-12.png" width=700 />

In [21]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1) # h 와 target_W 내적(dot)
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout): # TODO: 역전파 구현 생각해보기
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

### 4.2.5 네거티브 샘플링

`정답`을 입력하면 `Sigmoid` 출력값이 1,
`오답`을 입력하면 출력값이 0에 가까워야함  
이런 결과를 만들어주는 가중치가 필요


모든 부정적인 예를 대상으로 학습을 하면 어휘 수가 늘어났을 때 감당이 어렵다.  
그래서 적은 수(5개라든지, 10개라든지)의 부정적 예를 샘플링해 사용.  
이것이 바로 `네거티브 샘플링`

<p align="center"><img src="./master/images/fig 4-17.png" width=500 />

### 4.2.6 네거티브 샘플링의 샘플링 기법

말뭉치에서 자주 등장하는 단어를 많이 추출하고 드물게 등장하는 단어를 적게 추출하는 샘플링 기법을 사용

1. 말뭉치에서 각 단어의 출현 횟수를 구해 '확률 분포'로 나타낸다.
2. 그 확률분포대로 단어를 샘플링한다.

#### np.random.choice() 사용 예시

In [22]:
import numpy as np

np.random.choice(10)

6

In [23]:
np.random.choice(10)

6

In [24]:
# words에서 하나만 무작위로 샘플링
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'say'

In [25]:
# 5개만 무작위로 샘플링 (중복 있음)
np.random.choice(words, size=5)

array(['you', 'goodbye', 'hello', 'goodbye', 'say'], dtype='<U7')

In [26]:
# 5개만 무작위로 샘플링 (중복 없음)
np.random.choice(words, size=5, replace=False)

array(['I', 'hello', '.', 'say', 'goodbye'], dtype='<U7')

In [27]:
# 확률분포에 따라 샘플링
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'you'

In [28]:
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

<img src="./master/images/e 4-4.png" width=200 align='left' />  
  
  word2vec의 네거티브 샘플링에서는 **확률분포에 0.75를 제곱**하라고 권고   
  수정 후에도 총합은 1이 되어야 하므로, '수정 후 확률 분포의 총합'으로 나눠준다.  
  이렇게 0.75제곱을 해주는 이유는 **확률이 낮은 단어를 버리지 않기 위해서**

### 4.2.7 네거티브 샘플링 구현

In [30]:
# NegativeSamplingLoss 에 필요한 SigmoidWithLoss
from common.functions import cross_entropy_error

class SigmoidWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.loss = None
        self.y = None # sigmoid의 출력
        self.t = None # 정답 데이터
        
    def forward(self, x, t):
        self.t = t
        self.y = 1 / (1 + np.exp(-x))
        
        self.loss = cross_entropy_error(np.c_[1 - self.y, self.y], self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        
        dx = (self.y - self.t) * dout / batch_size
        return dx

In [31]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target) # TODO : target 이 뭔디?
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)
        
        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            

In [ ]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        '''
        :W: 가중치
        :corpus: 말뭉치(단어ID의 리스트)
        :power: 확률분포에 제곱할 값
        :sample_size: 부정적 예의 샘플링 횟수
        '''

# Chapter 4 보류